 In this notebook, we explore the utilization of Large Language Models (LLMs) to develop an advanced chatbot application. The dataset used for this project is sourced from car_reviews.csv, which contains sample car reviews and their associated sentiment labels. As a task for AI and NLP developer, our objective is to prototype a chatbot app equipped with multiple functionalities, including sentiment analysis, question answering, summarization, translation, and more.

The solution presented here leverages pre-trained Hugging Face LLMs to process textual prompts and generate responses for various tasks. By integrating state-of-the-art language processing techniques, we aim to create a versatile chatbot that can assist customers and support human agents efficiently. We will try to explore the implementation of LLMs to enhance customer service and support in the automotive domain.

In [1]:
!pip install transformers
!pip install evaluate
!pip install xformer
from transformers import logging
logging.set_verbosity(logging.WARNING)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 947.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00


In [19]:
from zipfile import ZipFile
import os

!unzip -o '/content/data.zip' -d '/content/extracted/'
extracted_files = os.listdir('/content/extracted/')


Archive:  /content/data.zip
  inflating: /content/extracted/data/car_reviews.csv  
  inflating: /content/extracted/data/reference_translations.txt  


In [4]:
import pandas as pd
import torch

# Load the car reviews dataset
file_path = "/content/extracted/data/car_reviews.csv"
df = pd.read_csv(file_path, delimiter=";")
display(df.head()), df.info(), df.describe()
# Put the car reviews and their associated sentiment labels in two lists
reviews = df['Review'].tolist()
real_labels = df['Class'].tolist()


,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Review  5 non-null      object
 1   Class   5 non-null      object
dtypes: object(2)
memory usage: 208.0+ bytes


Lets say. The CTO at "Car-ing is sharing", a car sales and rental company, hired you to help prototype a chatbot app that addresses diverse inquiries using LLMs.

## Instruction 1: sentiment classification
Use a pre-trained LLM to classify the sentiment of the five car reviews in the car_reviews.csv dataset, and evaluate the classification accuracy and F1 score of predictions.

In [6]:
# Load a pre-trained sentiment analysis model into a pipeline
# The 'pipeline' function from the Hugging Face Transformers library is used to load a pre-trained model.
# In this case, we're loading a sentiment analysis model fine-tuned on the SST-2 dataset.
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english')

# Perform sentiment analysis on the car reviews using the loaded model
# The 'classifier' pipeline is used to predict the sentiment of each review.
# Predictions are made for each review and stored in 'predicted_labels'.
predicted_labels = classifier(reviews)

# Each review, along with its actual sentiment and predicted sentiment, is printed to the console.
for review, prediction, label in zip(reviews, predicted_labels, real_labels):
    print(f"Review: {review}\nActual Sentiment: {label}\nPredicted Sentiment: {prediction['label']} (Confidence: {prediction['score']:.4f})\n")

# Custom evaluation metrics are loaded from the 'evaluate' module.
import evaluate
accuracy = evaluate.load("accuracy")
f1 = evaluate.load("f1")

# Actual and predicted sentiment labels are mapped to integer labels for computation.
references = [1 if label == "POSITIVE" else 0 for label in real_labels]
predictions = [1 if label['label'] == "POSITIVE" else 0 for label in predicted_labels]

# Step 6: Calculate accuracy and F1 score for the sentiment predictions
# The 'compute' method is used to calculate accuracy and F1 score based on actual and predicted labels.
accuracy_result_dict = accuracy.compute(references=references, predictions=predictions)
accuracy_result = accuracy_result_dict['accuracy']
f1_result_dict = f1.compute(references=references, predictions=predictions)
f1_result = f1_result_dict['f1']

# The calculated accuracy and F1 score are printed to the console.
print(f"Accuracy: {accuracy_result}")
print(f"F1 result: {f1_result}")


Review: I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel tr

## Instruction 2: Translation
The company is recently attracting customers from Spain. Extract and pass the first two sentences of the first review in the dataset to an English-to-Spanish translation LLM. Calculate the BLEU score to assess translation quality

In [12]:

# Load translation LLM into a pipeline and translate car review
first_review = reviews[0]
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-es")
translated_review = translator(first_review, max_length=400)[0]['translation_text']
print(f"Model translation:\n{translated_review}")

# Load reference translations from file
with open("/content/extracted/data/reference_translations.txt", 'r') as file:
    lines = file.readlines()
references = [line.strip() for line in lines]
print(f"Spanish translation references:\n{references}")

# Load and calculate BLEU score metric
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=[translated_review], references=[references])
print(bleu_score['bleu'])



Your input_length: 365 is bigger than 0.9 * max_length: 400. You might consider increasing your max_length manually, e.g. translator('...', max_length=400)


Model translation:
Estoy muy satisfecho con mi 2014 Nissan NV SL. Utilizo esta furgoneta para mis entregas de negocios y uso personal. Camping, viajes por carretera, etc. No tenemos ningún niño así que guardo la mayoría de los asientos en mi almacén. Quería la furgoneta de pasajeros para el aire acondicionado trasero. Hemos conducido nuestra camioneta de Florida a California para un viaje Cross Country en 2014. Promedimos alrededor de 18 mpg. Conducimos a través de una gran cantidad de lluvia y Era un vehículo muy cómodo y estable. El motor V8 Nissan Titan es un motor de 500k mile. Se ha probado muchas veces por entrega y compañías de camiones. Es por eso que Nissan le da un 5 año o 100k mile parachoques a la garantía de parachoques. Muchas personas tienen miedo de conducir esta furgoneta debido a su tamaño. Pero con sensores de sonar delanteros y traseros, grandes espejos y la cámara de respaldo. Es fácil conducir. Los sensores delanteros y traseros también monitorean los lados delant

0.031234303847171685


# Instruction 3: Extractive QA
The 2nd review in the dataset emphasizes brand aspects. Load an extractive QA LLM such as "deepset/minilm-uncased-squad2" to formulate the question "What did he like about the brand?" and obtain an answer.

In [14]:


# Import auto classes (optional: can be solved via pipelines too)
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering

# Instantiate model and tokenizer
model_ckp = "deepset/minilm-uncased-squad2"
tokenizer = AutoTokenizer.from_pretrained(model_ckp)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckp)

# Define context and question, and tokenize them
context = reviews[1]
print(f"Context:\n{context}")
question = "What did he like about the brand?"
inputs = tokenizer(question, context, return_tensors="pt")

# Perform inference and extract answer from raw outputs
with torch.no_grad():
  outputs = model(**inputs)
start_idx = torch.argmax(outputs.start_logits)
end_idx = torch.argmax(outputs.end_logits) + 1
answer_span = inputs["input_ids"][0][start_idx:end_idx]

# Decode and show answer
answer = tokenizer.decode(answer_span)
print("Answer: ", answer)

tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Context:
The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found.
Answer:  ride quality, reliability


## Instruction 4: Summarization
Summarize the last review in the dataset, into approximately 50-55 tokens long.

In [18]:


# Get original text to summarize upon car review
text_to_summarize = reviews[-1]
print(f"Original text:\n{text_to_summarize}")

# Load summarization pipeline and perform inference
model_name = "cnicu/t5-small-booksum"
summarizer = pipeline("summarization", model=model_name)
outputs = summarizer(text_to_summarize, max_length=53)
summarized_text = outputs[0]['summary_text']
print(f"Summarized text:\n{summarized_text}")

Original text:
I've been dreaming of owning an SUV for quite a while, but I've been driving cars that were already paid for during an extended period. I ultimately made the decision to transition to a brand-new car, which, of course, involved taking on new payments. However, given that I don't drive extensively, I was inclined to avoid a substantial financial commitment. The Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment; the financial arrangement is quite reasonable. Handling and styling are great; I have hauled 12 bags of mulch in the back with the seats down and could have held more. I am VERY satisfied overall. I find myself needing to exercise extra caution when making lane changes, particularly owing to the blind spots resulting from the small side windows situated towards the rear of the vehicle. To address this concern, I am actively engaged in making adjustments to my mirrors and consciously reducing the frequency of la